<a href="https://colab.research.google.com/github/ASM650/SADAIA_Bootcamp/blob/main/Saudi_Dialect_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [3]:
! pip install arabert
! pip install farasapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=96fbebbdf175be7ee6c03e416b99724d0b7b571b9b3ae3fc2c2dc53cac916848
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji


In [4]:
from arabert.preprocess import ArabertPreprocessor
model_name="bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:22<00:00, 10.6MiB/s]


[2023-09-22 14:46:13,789 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_df = pd.read_csv('/content/drive/MyDrive/Project_SDAIA/train_SADA.csv')
#train_df.sample(2)

val_df = pd.read_csv('/content/drive/MyDrive/Project_SDAIA/valid_SADA.csv')
#val_df.sample(2)

test_df = pd.read_csv('/content/drive/MyDrive/Project_SDAIA/test_SADA.csv')
#test_df.sample(2)

In [8]:
combined_df = pd.concat([train_df, val_df], axis=0)

In [9]:
# dropping null values
combined_df = combined_df.dropna()
test_df = test_df.dropna()
# I will drop the ProcessedText and clean the GroundTruthText instead
# combined_df.drop(['ProcessedText'], axis = 1, inplace = True)
# test_df.drop(['ProcessedText'], axis = 1, inplace = True)

In [10]:
combined_df.isnull().sum()

Unnamed: 0         0
FileName           0
ShowName           0
FullFileLength     0
SegmentID          0
SegmentLength      0
SegmentStart       0
SegmentEnd         0
SpeakerAge         0
SpeakerGender      0
SpeakerDialect     0
Speaker            0
Environment        0
GroundTruthText    0
ProcessedText      0
Category           0
dtype: int64

In [11]:
test_df.isnull().sum()

Unnamed: 0         0
FileName           0
ShowName           0
FullFileLength     0
SegmentID          0
SegmentLength      0
SegmentStart       0
SegmentEnd         0
SpeakerAge         0
SpeakerGender      0
SpeakerDialect     0
Speaker            0
Environment        0
GroundTruthText    0
ProcessedText      0
Category           0
dtype: int64

In [12]:
# just renaming
combined_df['SpeakerDialect'].replace(['More than 1 speaker اكثر من متحدث'],["More than one speaker"],inplace=True)
test_df['SpeakerDialect'].replace(['More than 1 speaker اكثر من متحدث'],["More than one speaker"],inplace=True)

In [13]:
# combien Najdi, Hijazi,Shamali and Janubi under the Saudi class
combined_df['SpeakerDialect'].replace(['Najdi'],["Saudi"],inplace=True)

combined_df['SpeakerDialect'].replace({'Hijazi':'Saudi'},inplace=True)
combined_df['SpeakerDialect'].replace({'Shamali':'Saudi'},inplace=True)
combined_df['SpeakerDialect'].replace({'Janubi':'Saudi'},inplace=True)

# all ther clasess will be Not Saudi
combined_df['SpeakerDialect'].replace({'Levantine':'Not Saudi'},inplace=True)

combined_df['SpeakerDialect'].replace({'Maghrebi':'Not Saudi'},inplace=True)
combined_df['SpeakerDialect'].replace({'Yemeni':'Not Saudi'},inplace=True)
combined_df['SpeakerDialect'].replace({'Egyptian':'Not Saudi'},inplace=True)
combined_df['SpeakerDialect'].replace({'Khaliji':'Not Saudi'},inplace=True)
combined_df['SpeakerDialect'].replace({'ModernStandardArabic':'Not Saudi'},inplace=True)
combined_df['SpeakerDialect'].replace({'Iraqi':'Not Saudi'},inplace=True)

# except Unknown,Notapplicable and More than one speaker
combined_df = combined_df.drop(combined_df[combined_df['SpeakerDialect'] == 'Unknown'].index)
combined_df = combined_df.drop(combined_df[combined_df['SpeakerDialect'] == 'Notapplicable'].index)
combined_df = combined_df.drop(combined_df[combined_df['SpeakerDialect'] == 'More than one speaker'].index)

In [14]:
combined_df.SpeakerDialect.value_counts()

Saudi        127030
Not Saudi     36315
Name: SpeakerDialect, dtype: int64

In [15]:
num_clasess = combined_df.SpeakerDialect.nunique()

In [16]:
# Same for test set
test_df['SpeakerDialect'].replace(['Najdi'],["Saudi"],inplace=True)
test_df['SpeakerDialect'].replace({'Hijazi':'Saudi'},inplace=True)
test_df['SpeakerDialect'].replace({'Shamali':'Saudi'},inplace=True)

test_df['SpeakerDialect'].replace({'Khaliji':'Not Saudi'},inplace=True)
test_df['SpeakerDialect'].replace({'Egyptian':'Not Saudi'},inplace=True)
test_df['SpeakerDialect'].replace({'Yemeni':'Not Saudi'},inplace=True)
test_df['SpeakerDialect'].replace({'ModernStandardArabic':'Not Saudi'},inplace=True)

test_df = test_df.drop(test_df[test_df['SpeakerDialect'] == 'Unknown'].index)
test_df = test_df.drop(test_df[test_df['SpeakerDialect'] == 'Notapplicable'].index)
test_df = test_df.drop(test_df[test_df['SpeakerDialect'] == 'More than one speaker'].index)

test_df.SpeakerDialect.value_counts()

Saudi        2530
Not Saudi    1410
Name: SpeakerDialect, dtype: int64

In [17]:
X_Arabert_tok = combined_df['ProcessedText'].apply(lambda x:arabert_prep.preprocess(x))
X_Araberttest_tok = test_df['ProcessedText'].apply(lambda x:arabert_prep.preprocess(x))

In [18]:
map_label={
    'Not Saudi':0,
    'Saudi':1,
}
label_map={
    0:'Not Saudi',
    1:'Saudi',
}

In [19]:
from sklearn.metrics import (accuracy_score, f1_score,recall_score)
from torch.utils.data import  Dataset
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                        AutoTokenizer, BertTokenizer, Trainer,
                        TrainingArguments)
from transformers.data.processors.utils import InputFeatures

In [20]:
model_name = 'aubmindlab/bert-base-arabert'
num_labels = num_clasess
max_len = 120

In [21]:
class ClassificationDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(ClassificationDataset).__init__()

      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map


    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

      inputs = self.tokenizer(
          text,
          max_length=self.max_len,
          padding='max_length',
          truncation=True
        )
      return InputFeatures(**inputs,label= self.target[item])

In [22]:
labels = combined_df['SpeakerDialect'].map(map_label)
test_labels = combined_df['SpeakerDialect'].map(map_label)

In [23]:
labels = labels.astype(int)
test_labels = test_labels.astype(int)

In [24]:
train_dataset = ClassificationDataset(
    X_Arabert_tok.to_list(),
    labels.to_list(),
    model_name,
    max_len,
    map_label
)


test_dataset = ClassificationDataset(
    X_Araberttest_tok.to_list(),
    test_labels.to_list(),
    model_name,
    max_len,
    map_label
)

In [25]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=num_labels)

In [26]:
def compute_metrics(p):
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'accuracy': acc,
      'recall':macro_recall
  }

In [27]:
training_args = TrainingArguments(
    output_dir= "./train",
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    fp16 = True,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    gradient_accumulation_steps = 4,
    num_train_epochs= 5,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit=10,
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'eval_loss',
    greater_is_better = False,
    report_to=[]
  )

In [28]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [29]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Macro F1,Accuracy,Recall
0,0.492800,0.599510,0.458236,0.718528,0.512188
1,0.450800,0.637568,0.495188,0.704822,0.521933
2,0.415400,0.681230,0.493025,0.700254,0.519044
4,0.377100,0.731490,0.516963,0.680711,0.526136
4,0.351600,0.767186,0.512904,0.686041,0.525090


TrainOutput(global_step=6380, training_loss=0.41938304123833636, metrics={'train_runtime': 4944.699, 'train_samples_per_second': 165.172, 'train_steps_per_second': 1.29, 'total_flos': 5.0355755996688e+16, 'train_loss': 0.41938304123833636, 'epoch': 5.0})